Zadanie #2 

1. Korzystając z biblioteki sqlite3 utwórz nową bazę danych z tabelą FlightLeg, która będzie zawierała informację o lotach samolotów, z następującymi kolumnami: 

id — identyfikator numeryczny nadawany z sekwencji 

tailNumber — identyfikator samolotu 

sourceAirportCode, destinationAirportCode — trzyliterowy kod lotniska (wg IATA) 

sourceCountryCode, destinationCountryCode — trzyliterowy kod kraju (wg ISO 3166-1 Alpha-3) 

departureTimeUtc, landingTimeUtc — data i czas (z dokładnością do sekundy) odpowiednio odlotu i lądowania samolotu (w UTC) 

2. Wypełnij tabelę FlightLeg danymi z pliku: https://bitpeak.pl/datasets/flightlegs.csv

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('flightlegs.csv',
                 sep=';'
                )
df.head()

,tailNumber,source_airport_code,source_country_code,destination_airport_code,destination_country_code,departure_time,landing_time
0,A6-EUC,DXB,ARE,CAI,EGY,2021-11-30 04:47:00,2021-11-30 08:25:00
1,A6-ENE,ACC,GHA,ABJ,CIV,2021-11-29 13:35:00,2021-11-29 14:17:00
2,A6-END,DXB,ARE,MNL,PHL,2021-11-27 23:50:11,2021-11-28 07:58:00
3,A6-EPD,IST,TUR,DXB,ARE,2021-11-28 17:51:53,2021-11-28 21:31:00
4,A6-EUU,JED,SAU,DXB,ARE,2021-11-28 08:53:00,2021-11-28 11:10:00


In [3]:
#adding id column manually 
df.insert(0, 'id', df.index)
df.head()

,id,tailNumber,source_airport_code,source_country_code,destination_airport_code,destination_country_code,departure_time,landing_time
0,0,A6-EUC,DXB,ARE,CAI,EGY,2021-11-30 04:47:00,2021-11-30 08:25:00
1,1,A6-ENE,ACC,GHA,ABJ,CIV,2021-11-29 13:35:00,2021-11-29 14:17:00
2,2,A6-END,DXB,ARE,MNL,PHL,2021-11-27 23:50:11,2021-11-28 07:58:00
3,3,A6-EPD,IST,TUR,DXB,ARE,2021-11-28 17:51:53,2021-11-28 21:31:00
4,4,A6-EUU,JED,SAU,DXB,ARE,2021-11-28 08:53:00,2021-11-28 11:10:00


3. Dodaj do tabeli dwie nowe kolumny: 

flightDuration — wypełnij ją wartościami określającymi czas trwania lotu w minutach (zaokrągl wartości do najbliższej liczby całkowitej) 

flightType — wypełnij ją wartościami określającymi typ lotu: krajowy (wartość ‘D’ = domestic) lub zagraniczny (wartość ‘I’ = international); lot krajowy to lot zaczynający się i kończący w tym samym kraju 

In [4]:
#changing string objects into datetime objects
df['departure_time'] =  pd.to_datetime(df['departure_time'])
df['landing_time'] =  pd.to_datetime(df['landing_time'])

#calculating delta and converting it into minutes (rounded value)
df['flightDuration'] = df['landing_time']-df['departure_time']

df['flightDuration'] = df['flightDuration'].apply(lambda td: round(td.seconds/60))

In [5]:
def dom_or_int(source,destination):
    if source == destination:
        return 'D'
    else:
        return 'I'

In [6]:
df['flightType'] = df.apply(lambda x: dom_or_int(x.source_country_code, x.destination_country_code), axis=1)

4. Zaimplementuj logikę odpowiadającą na poniższe pytania: 

    Który samolot wykonał najwięcej lotów? 

    Który samolot przeleciał najwięcej minut? 

    Który lot, w podziale na krajowe i zagraniczne, był najkrótszy, a który najdłuższy, i ile minut trwał? 

    (punkt opcjonalny) Ile jest błędnych rekordów opisujących loty, które wskazują, że samolot wykonywał więcej niż jeden lot jednocześnie? Wyświetl wszystkie pary takich konfliktowych lotów. 

    (punkt opcjonalny) Odstęp pomiędzy którymi lotami tego samego dowolnego samolotu był najkrótszy i ile minut trwał? 

    Uwaga: Lot identyfikowany jest przez parę wartości tailNumber oraz departureTimeUtc. 

In [7]:
# 1 Który samolot wykonał najwięcej lotów?

df['tailNumber'].value_counts().idxmax()

'A6-ENS'

In [8]:
# 2 Który samolot przeleciał najwięcej minut?

df.groupby('tailNumber').sum()['flightDuration'].idxmax()

'A6-EVF'

In [9]:
# 3 Który lot, w podziale na krajowe i zagraniczne, był najkrótszy, a który najdłuższy, i ile minut trwał?

In [10]:
print('najkrótszy krajowy lot')
print('tailNumber: ',df[df['flightType']=='D'].sort_values(by='flightDuration',ascending=True).iloc[0]['tailNumber'])
print('departure_time ',df[df['flightType']=='D'].sort_values(by='flightDuration',ascending=True).iloc[0]['departure_time'])
print('czas trwania w minutach: ', df[df['flightType']=='D'].sort_values(by='flightDuration',ascending=True).iloc[0]['flightDuration'])

najkrótszy krajowy lot
tailNumber:  A6-ECH
departure_time  2021-11-30 09:32:00
czas trwania w minutach:  61


In [11]:
print('najdłuższy krajowy lot')
print('tailNumber: ',df[df['flightType']=='D'].sort_values(by='flightDuration',ascending=False).iloc[0]['tailNumber'])
print('departure_time ',df[df['flightType']=='D'].sort_values(by='flightDuration',ascending=False).iloc[0]['departure_time'])
print('czas trwania w minutach: ', df[df['flightType']=='D'].sort_values(by='flightDuration',ascending=False).iloc[0]['flightDuration'])

najdłuższy krajowy lot
tailNumber:  A6-EES
departure_time  2021-11-30 09:20:00
czas trwania w minutach:  126


In [12]:
print('najkrótszy międzynarodowy lot')
print('tailNumber: ',df[df['flightType']=='I'].sort_values(by='flightDuration',ascending=True).iloc[0]['tailNumber'])
print('departure_time ',df[df['flightType']=='I'].sort_values(by='flightDuration',ascending=True).iloc[0]['departure_time'])
print('czas trwania w minutach: ', df[df['flightType']=='I'].sort_values(by='flightDuration',ascending=True).iloc[0]['flightDuration'])


najkrótszy międzynarodowy lot
tailNumber:  A6-ENE
departure_time  2021-11-29 13:35:00
czas trwania w minutach:  42


In [13]:
print('najdłuższy międzynarodowy lot')
print('tailNumber: ',df[df['flightType']=='I'].sort_values(by='flightDuration',ascending=False).iloc[0]['tailNumber'])
print('departure_time ',df[df['flightType']=='I'].sort_values(by='flightDuration',ascending=False).iloc[0]['departure_time'])
print('czas trwania w minutach: ', df[df['flightType']=='I'].sort_values(by='flightDuration',ascending=False).iloc[0]['flightDuration'])

najdłuższy międzynarodowy lot
tailNumber:  A6-EWH
departure_time  2021-11-29 06:32:00
czas trwania w minutach:  966


4. (punkt opcjonalny) Ile jest błędnych rekordów opisujących loty, które wskazują, że samolot wykonywał więcej niż jeden lot jednocześnie? Wyświetl wszystkie pary takich konfliktowych lotów.

In [14]:
#converting datestamps to datetime object

In [15]:
from datetime import datetime
df['departure_time'] = df['departure_time'].apply(lambda ts:datetime.strptime(str(ts), "%Y-%m-%d %H:%M:%S")) 

In [16]:
df['landing_time'] = df['landing_time'].apply(lambda ts:datetime.strptime(str(ts), "%Y-%m-%d %H:%M:%S")) 

In [17]:
def collision_check(df):
    
    #creating a list of colisions
    fligts_colision = []

    
    for plane in df['tailNumber'].unique():

        df_tN = df[df['tailNumber']==plane][['id','tailNumber','departure_time','landing_time']]
        num_of_flights = df_tN.shape[0]

        #for every flight
        for i in range(num_of_flights-1):
            dept_time_i = df_tN['departure_time'].iloc[i]
            land_time_i = df_tN['landing_time'].iloc[i]
            flight_id_i = df_tN['id'].iloc[i]
            
            #checking all other flights 
            for j in range(i+1,num_of_flights-1):
                dept_time_j = df_tN['departure_time'].iloc[j]
                land_time_j = df_tN['landing_time'].iloc[j]
                flight_id_j = df_tN['id'].iloc[j]

                # if a flight 'j' started or landed during other flight 'i' (after i started and before i landed)
                # both numbers are added to 'fligts_colision' list as a list
                if dept_time_i <= dept_time_j <= land_time_i or dept_time_i <= land_time_j <= land_time_i:
                    fligts_colision.append([flight_id_i,flight_id_j])
                else:
                    pass

    return fligts_colision

In [18]:
collision_check(df)

[[384, 499]]

In [19]:
#check- flight 499 departed before 384 landed: 

In [20]:
df.iloc[384]

id                                          384
tailNumber                               A6-ECS
source_airport_code                         ATH
source_country_code                         GRC
destination_airport_code                    DXB
destination_country_code                    ARE
departure_time              2021-11-30 15:45:00
landing_time                2021-11-30 19:59:00
flightDuration                              254
flightType                                    I
Name: 384, dtype: object

In [21]:
df.iloc[499]

id                                          499
tailNumber                               A6-ECS
source_airport_code                         EWR
source_country_code                         USA
destination_airport_code                    ATH
destination_country_code                    GRC
departure_time              2021-11-30 05:14:44
landing_time                2021-11-30 15:50:00
flightDuration                              635
flightType                                    I
Name: 499, dtype: object

5. (punkt opcjonalny) Odstęp pomiędzy którymi lotami tego samego dowolnego samolotu był najkrótszy i ile minut trwał? 

In [22]:
def min_pause(df, plane):
    
    #choosing flights for given plane and sorting chronologically
    df_tN = df[df['tailNumber']==plane][['id','tailNumber','departure_time','landing_time']]
    df_tN.sort_values(by='departure_time',ascending=True, inplace=True)
    
    #new columns with landing time of the previous flight and its id
    df_tN['previous_landing_time'] = df_tN['landing_time'].shift(1)
    
    df_tN['id'] = df_tN['id'].apply(lambda x:str(x))
    df_tN['previous_id'] = df_tN['id'].shift(1)

    #calculating pause and converting into minutes
    df_tN['pause_in_minutes'] = df['departure_time']-df_tN['previous_landing_time']
    df_tN['pause_in_minutes'].iloc[1:] = df_tN['pause_in_minutes'].iloc[1:].apply(lambda td:round(td.seconds/60))

    #choosing the shortest pause and corresponding flights
    shortest_pause_duration = df_tN['pause_in_minutes'].min()
    id_after_shortest_pause = df_tN[df_tN['pause_in_minutes']==shortest_pause_duration]['id'].values[0]
    id_before_shortest_pause = df_tN[df_tN['pause_in_minutes']==shortest_pause_duration]['previous_id'].values[0]

    #printing the output
    print(f'The shortest pause for the aeroplane {plane} was after flight no. {id_before_shortest_pause}, ', end='')  
    print(f'before flight no. {id_after_shortest_pause} and lasted for {shortest_pause_duration} minutes')

In [24]:
min_pause(df,'A6-ECS')

The shortest pause for the aeroplane A6-ECS was after flight no. 349, before flight no. 206 and lasted for 150 minutes
